<a href="https://colab.research.google.com/github/xtenix88/Data-Portfolio/blob/main/Investment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as no
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import time
import os

In [5]:
pip install yfinance

     |████████████████████████████████| 6.4 MB 7.7 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [6]:
import yfinance as yf

In [82]:
# Start end date defaults
S_DATE = "2017-02-01"
E_DATE = "2023-02-01"
S_DATE_DT = pd.to_datetime(S_DATE)
E_DATE_DT = pd.to_datetime(E_DATE)

In [7]:
stock = yf.Ticker("AAPL")
df = stock.history(period="5y")
df.sort_index(axis = 0.,  ascending=False)


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-02-01,174.009995,174.589996,172.320007,173.050003,46551332,0.0,0.0
2022-01-31,170.160004,175.000000,169.509995,174.779999,115307800,0.0,0.0
2022-01-28,165.710007,170.350006,162.800003,170.330002,179485800,0.0,0.0
2022-01-27,162.449997,163.839996,158.279999,159.220001,116691400,0.0,0.0
2022-01-26,163.500000,164.389999,157.820007,159.690002,108275300,0.0,0.0
...,...,...,...,...,...,...,...
2017-02-07,30.718732,31.083479,30.697554,30.951700,152735200,0.0,0.0
2017-02-06,30.386922,30.709310,30.332796,30.659891,107383600,0.0,0.0
2017-02-03,30.193972,30.401055,30.158675,30.375170,98029200,0.0,0.0


In [8]:
def get_column_from_csv(file, col_name):
    # Try to get the file and if it doesn't exist issue a warning
    try:
        df = pd.read_csv(file)
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df[col_name]

In [21]:
tickers = get_column_from_csv("/content/Wilshire-5000-Stocks.csv", "Ticker")
print(len(tickers))
tickers

5


0    AMZN
1    COST
2     DIS
3    DOCU
4    TSLA
Name: Ticker, dtype: object

In [10]:
def save_to_csv_from_yahoo(folder, ticker):
    stock = yf.Ticker(ticker)
    
    try:
        print("Get Data for : ", ticker)
        # Get historical closing price data
        df = stock.history(period="5y")
    
        # Wait 2 seconds
        time.sleep(2)
        
        # Remove the period for saving the file name
        # Save data to a CSV file
        # File to save to 
        the_file = folder + ticker.replace(".", "_") + '.csv'
        print(the_file, " Saved")
        df.to_csv(the_file)
    except Exception as ex:
        print("Couldn't Get Data for :", ticker)

In [11]:
PATH = "/content/"

In [12]:
for x in range(0, 6):
    save_to_csv_from_yahoo(PATH, tickers[x])
    print("Finished")

Get Data for :  AMZN
/content/AMZN.csv  Saved
Finished
Get Data for :  COST
/content/COST.csv  Saved
Finished
Get Data for :  DIS
/content/DIS.csv  Saved
Finished
Get Data for :  DOCU
/content/DOCU.csv  Saved
Finished
Get Data for :  TSLA
/content/TSLA.csv  Saved
Finished


KeyError: ignored

In [72]:
def get_stock_df_from_csv(tickers):
  try:
    df = pd.read_csv(PATH + tickers + '.csv', index_col=0)
  except FileNotFoundError:
    print("File Doesn't Exist")
  else:
    return df

In [73]:
get_stock_df_from_csv('AMZN')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-02-01,829.210022,833.780029,824.940002,832.349976,3850200,0,0
2017-02-02,836.590027,842.489990,828.260010,839.950012,7350500,0,0
2017-02-03,806.719971,818.299988,804.000000,810.200012,10868800,0,0
2017-02-06,809.799988,810.719971,803.000000,807.640015,3897300,0,0
2017-02-07,809.309998,816.159973,807.500000,812.500000,3466100,0,0
...,...,...,...,...,...,...,...
2022-01-26,2895.000000,2903.699951,2746.370117,2777.449951,4780100,0,0
2022-01-27,2816.000000,2884.870117,2787.000000,2792.750000,3860100,0,0
2022-01-28,2817.209961,2879.959961,2758.590088,2879.560059,3705700,0,0


In [40]:
tickers

0    AMZN
1    COST
2     DIS
3    DOCU
4    TSLA
Name: Ticker, dtype: object

In [14]:
from os import listdir
from os.path import isfile, join

files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
tickers

['Wilshire-5000-Stocks', 'AMZN', 'TSLA', 'DIS', 'DOCU', 'COST']

In [15]:
def add_daily_return_to_df(df):
  df['daily_return'] = (df['Close'] / df['Close'].shift(1)) - 1
  return df

In [16]:
def add_cum_return_to_df(df):
  df['cum_return'] = (1 + df['daily_return']).cumprod()
  return df

In [17]:
def add_bollinger_bands(df):
  df['middle_band'] = df['Close'].rolling(window=20).mean()
  df['upper_band'] = df['middle_band'] + 1.96 * df['Close'].rolling(window=20).std()
  df['lower_band'] = df['middle_band'] - 1.96 * df['Close'].rolling(window=20).std()
  return df

In [56]:
def add_ichimoku(df):
  #Conversion line
  hi_val = df['High'].rolling(window=9).max()
  lo_val = df['Low'].rolling(window=9).min()
  df['Conversion'] = (hi_val+lo_val)/2

  #Base Line
  hi_val2 = df['High'].rolling(window=26).max()
  lo_val2 = df['Low'].rolling(window=26).min()
  df['Baseline'] = (hi_val2+lo_val2)/2

  #Span A
  df['SpanA'] = ((df['Conversion'] + df['Baseline']) / 2)

  #Span B
  hi_val3 = df['High'].rolling(window=52).max()
  lo_val3 = df['Low'].rolling(window=52).min()
  df['SpanB'] = ((hi_val3 + lo_val3)/2).shift(26)

  #Laggine Span
  df['Lagging'] = df['Close'].shift(-26)
  return df



In [74]:
for x in tickers:
  try:
    print("Working on:", x)
    new_df = get_stock_df_from_csv(x)
    new_df = add_daily_return_to_df(new_df)
    new_df = add_cum_return_to_df(new_df)
    new_df = add_bollinger_bands(new_df)
    new_df = add_ichimoku(new_df)
    new_df.to_csv(PATH + x + '.csv')
  except Exception as ex:
    print(ex)

Working on: AMZN
Working on: COST
Working on: DIS
Working on: DOCU
Working on: TSLA


In [59]:
df = pd.read_csv('/content/AMZN.csv', index_col=0)

#daily return
df['daily_return'] = (df['Close'] / df['Close'].shift(1)) - 1

#cumulative return
df['cum_return'] = (1 + df['daily_return']).cumprod()

#moving average 20 days
df['middle_band'] = df['Close'].rolling(window=20).mean()
df['upper_band'] = df['middle_band'] + 1.96 * df['Close'].rolling(window=20).std()
df['lower_band'] = df['middle_band'] - 1.96 * df['Close'].rolling(window=20).std()

#Conversion line
hi_val = df['High'].rolling(window=9).max()
lo_val = df['Low'].rolling(window=9).min()
df['Conversion'] = (hi_val+lo_val)/2

#Base Line
hi_val2 = df['High'].rolling(window=26).max()
lo_val2 = df['Low'].rolling(window=26).min()
df['Baseline'] = (hi_val2+lo_val2)/2

#Span A
df['SpanA'] = ((df['Conversion'] + df['Baseline']) / 2)

#Span B
hi_val3 = df['High'].rolling(window=52).max()
lo_val3 = df['Low'].rolling(window=52).min()
df['SpanB'] = ((hi_val3 + lo_val3)/2).shift(26)

#Laggine Span
df['Lagging'] = df['Close'].shift(-26)

#return df
df


,Open,High,Low,Close,Volume,Dividends,Stock Splits,daily_return,cum_return,middle_band,upper_band,lower_band,Conversion,Baseline,SpanA,SpanB,Lagging
Date,,,,,,,,,,,,,,,,,
2017-02-01,829.210022,833.780029,824.940002,832.349976,3850200,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,852.460022
2017-02-02,836.590027,842.489990,828.260010,839.950012,7350500,0,0,0.009131,1.009131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,854.590027
2017-02-03,806.719971,818.299988,804.000000,810.200012,10868800,0,0,-0.035419,0.973389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,852.530029
2017-02-06,809.799988,810.719971,803.000000,807.640015,3897300,0,0,-0.003160,0.970313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,852.969971
2017-02-07,809.309998,816.159973,807.500000,812.500000,3466100,0,0,0.006018,0.976152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,853.419983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-26,2895.000000,2903.699951,2746.370117,2777.449951,4780100,0,0,-0.007954,3.336878,3180.990503,3575.342296,2786.638710,3015.734985,3082.950073,3049.342529,3480.385010,NaN
2022-01-27,2816.000000,2884.870117,2787.000000,2792.750000,3860100,0,0,0.005509,3.355259,3151.427002,3568.704813,2734.149191,2976.020020,3082.950073,3029.485046,3499.214966,NaN
2022-01-28,2817.209961,2879.959961,2758.590088,2879.560059,3705700,0,0,0.031084,3.459554,3126.760510,3547.103219,2706.417802,2950.864990,3082.950073,3016.907532,3499.214966,NaN


In [78]:
def plot_with_boll_bands(df, ticker):
  fig = go.Figure()

  candle = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name='Candlestick')

  upper_line = go.Scatter(x=df.index, y=df['upper_band'], 
                          line=dict(color='rgba(250,0,0,0.75)',
                          width=1), name='Upper Band')

  mid_line = go.Scatter(x=df.index, y=df['middle_band'], 
                          line=dict(color='rgba(0,0,250,0.75)',
                          width=1), name='Middle Band')
  
  lower_line = go.Scatter(x=df.index, y=df['lower_band'], 
                          line=dict(color='rgba(0,25,0,0.75)',
                          width=1), name='Lower Band')
  
  fig.add_trace(candle)
  fig.add_trace(upper_line)
  fig.add_trace(mid_line)
  fig.add_trace(lower_line)

  fig.update_xaxes(title='Date', rangeslider_visible=True)
  fig.update_yaxes(title='Price')

  fig.update_layout(title="Bollinger Bands", height=1200, width=1800, showlegend=True)

  fig.show()

In [76]:
import plotly.graph_objs as go

fig = go.Figure()

candle = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name='Candlestick')

upper_line = go.Scatter(x=df.index, y=df['upper_band'], 
                          line=dict(color='rgba(250,0,0,0.75)',
                          width=1), name='Upper Band')

mid_line = go.Scatter(x=df.index, y=df['middle_band'], 
                          line=dict(color='rgba(0,0,250,0.75)',
                          width=1), name='Middle Band')
  
lower_line = go.Scatter(x=df.index, y=df['lower_band'], 
                          line=dict(color='rgba(0,25,0,0.75)',
                          width=1), name='Lower Band')
  
fig.add_trace(candle)
fig.add_trace(upper_line)
fig.add_trace(mid_line)
fig.add_trace(lower_line)

fig.update_xaxes(title='Date', rangeslider_visible=True)
fig.update_yaxes(title='Price')

fig.update_layout(title="Bollinger Bands", height=1200, width=1800, showlegend=True)

fig.show()

In [79]:
costco = get_stock_df_from_csv('COST')
plot_with_boll_bands(costco, 'COST')

In [98]:
def get_fill_color(label):
    if label >= 1:
        return 'rgba(0,250,0,0.4)'
    else:
        return 'rgba(250,0,0,0.4)'

In [89]:
import numpy as np

In [96]:
def get_ichimoku(df):
  candle = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name='Candlestick')

  df1 = df.copy()
  fig = go.Figure()
  df['label'] = np.where(df['SpanA'] > df['SpanB'], 1, 0) #return 1 for green, return 0 for red
  df['group'] = df['label'].ne(df['label'].shift()).cumsum()
  df = df.groupby('group')

  dfs = []
  for name, data in df:
    dfs.append(data)
  for df in dfs:
    fig.add_traces(go.Scatter(x=df.index, y=df.SpanA,
        line=dict(color='rgba(0,0,0,0)')))

    fig.add_traces(go.Scatter(x=df.index, y=df.SpanB,
        line=dict(color='rgba(0,0,0,0)'),
        fill='tonexty',
        fillcolor=get_fill_color(df['label'].iloc[0])))
    
  baseline = go.Scatter(x=df1.index, y=df1['Baseline'],
                        line=dict(color='pink', width=2), name='Baseline')
  
  conversion = go.Scatter(x=df1.index, y=df1['Conversion'],
                          line=dict(color='black', width=1), name='Conversion')
  
  lagging = go.Scatter(x=df1.index, y=df1['Lagging'],
                          line=dict(color='purple', width=2), name='Lagging')
  
  span_a = go.Scatter(x=df1.index, y=df1['SpanA'],
                          line=dict(color='green', width=2, dash='dot'), name='Span A')
  
  span_b = go.Scatter(x=df1.index, y=df1['SpanB'],
                          line=dict(color='red', width=1, dash='dot'), name='Span B')
  
  fig.add_trace(candle)
  fig.add_trace(baseline)
  fig.add_trace(conversion)
  fig.add_trace(lagging)
  fig.add_trace(span_a)
  fig.add_trace(span_b)

  fig.update_layout(height=1200, width=1800, showlegend=True)

  fig.show()



In [99]:
costco = get_stock_df_from_csv('COST')
get_ichimoku(costco)